## Support Vector Machine

### Setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)
# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

### Read Data

In [2]:
import pandas as pd

CSV_FILENAME = "steel_plate.csv" 
CSV_FOLDER_NAME = "Datasets"
CSV_FILEPATH = os.path.join(CSV_FOLDER_NAME, CSV_FILENAME)

try:
    raw_data = pd.read_csv(CSV_FILEPATH)
except Exception as e:
    print(e)

raw_data.head()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1


### Structure Data

In [18]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Fjern ID-kolonnen
raw_data_drop_id = raw_data.drop(columns=["id"])

# Slå sammen Pastry og Dirtiness til Other_Faults
raw_data_drop_id["Other_Faults"] = raw_data_drop_id["Pastry"] + raw_data_drop_id["Dirtiness"]

# Fjern de originale Pastry og Dirtiness kolonnene
raw_data_drop_id.drop(columns=["Pastry", "Dirtiness"], inplace=True)

# Definer features og target-kolonner
targets = ["Z_Scratch", "K_Scatch", "Stains", "Other_Faults", "Bumps"]
features = [col for col in raw_data_drop_id.columns if col not in targets]

# Velg hvilke features som ikke skal skaleres
exclude_scaling = ["TypeOfSteel_A300", "TypeOfSteel_A400"]
columns_to_scale = [col for col in features if col not in exclude_scaling]

# Skalering av features
scaler = StandardScaler()
scaled_data = raw_data_drop_id.copy()
scaled_data[columns_to_scale] = scaler.fit_transform(scaled_data[columns_to_scale])

# Konverter y fra én-hot-koding til en enkelt numerisk label
scaled_data["Defect_Type"] = scaled_data[targets].idxmax(axis=1)  # Henter kolonnen med verdi 1
scaled_data.drop(columns=targets, inplace=True)  # Fjerner én-hot-kodede kolonner

# Konverter labels til numerisk format
label_encoder = LabelEncoder()
scaled_data["Defect_Type"] = label_encoder.fit_transform(scaled_data["Defect_Type"])

# Splitt i features (X) og target (y)
X = scaled_data[features]
y = scaled_data["Defect_Type"]


### Split into train and test

In [19]:
from sklearn.model_selection import train_test_split

# Splitt i train/test-sett
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Sjekk at alt er riktig
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(y.value_counts())  # Ser på fordelingen av defekttyper

(15375, 27) (3844, 27) (15375,) (3844,)
Defect_Type
4    8508
0    4761
1    3432
2    1950
3     568
Name: count, dtype: int64


### One vs All - Training Modell

#### Linear

In [20]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Trene SVM med One-vs-All (default i SVC)
svm_model = SVC(kernel="linear", decision_function_shape="ovr", random_state=42)

# Tren modellen
svm_model.fit(X_train, y_train)

# Prediksjon
y_pred = svm_model.predict(X_test)

# Evaluer modellen
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

print(f"🔹 SVM One-vs-All Accuracy: {accuracy:.4f}")
print(report)

🔹 SVM One-vs-All Accuracy: 0.5965
              precision    recall  f1-score   support

       Bumps       0.00      0.00      0.00       952
    K_Scatch       0.89      0.87      0.88       686
Other_Faults       0.00      0.00      0.00       390
      Stains       0.76      0.78      0.77       114
   Z_Scratch       0.53      0.95      0.68      1702

    accuracy                           0.60      3844
   macro avg       0.44      0.52      0.47      3844
weighted avg       0.41      0.60      0.48      3844



c:\Users\Bruker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Bruker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Bruker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Polynominal - 3rd degree

In [21]:
# Trene SVM med One-vs-All (default i SVC)
svm_model = SVC(kernel="poly", degree=3, decision_function_shape="ovr", random_state=42)

# Tren modellen
svm_model.fit(X_train, y_train)

# Prediksjon
y_pred = svm_model.predict(X_test)

# Evaluer modellen
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

print(f"🔹 SVM One-vs-All Accuracy: {accuracy:.4f}")
print(report)

🔹 SVM One-vs-All Accuracy: 0.6220
              precision    recall  f1-score   support

       Bumps       0.59      0.25      0.35       952
    K_Scatch       0.89      0.87      0.88       686
Other_Faults       0.56      0.06      0.10       390
      Stains       0.75      0.75      0.75       114
   Z_Scratch       0.55      0.85      0.67      1702

    accuracy                           0.62      3844
   macro avg       0.67      0.56      0.55      3844
weighted avg       0.63      0.62      0.57      3844



#### Gaussian RBF

In [22]:
svm_rbf = SVC(kernel="rbf", C=1.0, gamma="scale", decision_function_shape="ovr", random_state=42)
svm_rbf.fit(X_train, y_train)

y_pred_rbf = svm_rbf.predict(X_test)

accuracy_rbf = accuracy_score(y_test, y_pred_rbf)
report_rbf = classification_report(y_test, y_pred_rbf, target_names=label_encoder.classes_)

print(f"🔹 SVM (RBF) Accuracy: {accuracy_rbf:.4f}")
print(report_rbf)

🔹 SVM (RBF) Accuracy: 0.6241
              precision    recall  f1-score   support

       Bumps       0.57      0.30      0.39       952
    K_Scatch       0.88      0.88      0.88       686
Other_Faults       0.61      0.05      0.09       390
      Stains       0.76      0.76      0.76       114
   Z_Scratch       0.56      0.83      0.67      1702

    accuracy                           0.62      3844
   macro avg       0.68      0.56      0.56      3844
weighted avg       0.63      0.62      0.58      3844



### Tuning - Gaussian RBF

In [23]:
from sklearn.model_selection import GridSearchCV

# Definerer hyperparameterrutenettet (grid) for C og gamma
param_grid = {
    'C': [0.1, 1, 10, 100],  # færre verdier
    'gamma': [0.001, 0.01, 0.1]  # færre verdier
}


# SVM-modell med RBF-kjerne
svm_rbf = SVC(kernel="rbf", decision_function_shape="ovr", random_state=42)

# GridSearchCV for å finne de beste parameterne
grid_search = GridSearchCV(svm_rbf, param_grid, cv=3, scoring="accuracy", verbose=2, n_jobs=-1)

# Kjør GridSearch på treningsdataene
grid_search.fit(X_train, y_train)

# Beste parametere og beste modell
print("Beste hyperparametere:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# Prediksjoner med den beste modellen
y_pred_best = best_model.predict(X_test)

# Beregn og vis ytelsen
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"🔹 Tuned SVM (RBF) Accuracy: {accuracy_best:.4f}")


Fitting 3 folds for each of 12 candidates, totalling 36 fits
Beste hyperparametere: {'C': 1, 'gamma': 0.1}
🔹 Tuned SVM (RBF) Accuracy: 0.6316


In [24]:
svm_rbf = SVC(kernel="rbf", C=1, gamma=0.1, decision_function_shape="ovr", random_state=42)
svm_rbf.fit(X_train, y_train)

y_pred_rbf = svm_rbf.predict(X_test)

accuracy_rbf = accuracy_score(y_test, y_pred_rbf)
report_rbf = classification_report(y_test, y_pred_rbf, target_names=label_encoder.classes_)

print(f"🔹 SVM (RBF) Accuracy: {accuracy_rbf:.4f}")
print(report_rbf)

🔹 SVM (RBF) Accuracy: 0.6316
              precision    recall  f1-score   support

       Bumps       0.57      0.39      0.46       952
    K_Scatch       0.89      0.87      0.88       686
Other_Faults       0.55      0.11      0.18       390
      Stains       0.74      0.75      0.75       114
   Z_Scratch       0.57      0.78      0.66      1702

    accuracy                           0.63      3844
   macro avg       0.66      0.58      0.59      3844
weighted avg       0.63      0.63      0.60      3844

